在每个复合体的top分子中采样zinc id，根据zinc id在结构文件中找到相应的pdbqt结构，聚合后输出。

In [ ]:
import os
import sys

sys.path.append("/home/huabei/projects/SMTarRNA")
os.chdir("/mnt/f/SMTarRNA_total_results/total_dataset_results/total_dataset_results/top_0_001")

os.listdir()

## 读取所有top分子的pdbqt结构

In [ ]:
import numpy as np

from src.dock_utils.utils import ZincPdbqt, write_pdbqt_to_gz


def read_zinc_id(file):
    zinc_ids = np.loadtxt(file, delimiter=",")[:, 0].tolist()
    zinc_ids = [int(zinc_id) for zinc_id in zinc_ids]
    return zinc_ids

In [ ]:
zinc_id_pdbqt = ZincPdbqt("four_complex_zinc_top_data_pdbqt.pdbqt.gz")

In [ ]:
from collections import defaultdict

from tqdm import tqdm

zinc_id_total = [int(z[4:]) for z in zinc_id_pdbqt.zinc_id]
zinc_id_total_dict = dict()
# 将zinc_id转换为索引，方便索引结构
for i, zinc_id in tqdm(enumerate(zinc_id_total)):
    zinc_id_total_dict[zinc_id] = i

## 索引并采样top分子

In [ ]:
# 索引top分子的zinc id
complexes = ["3a6p", "4z4d", "4z4c", "6cbd"]
complex_zinc_ids = [read_zinc_id(f"{complex}_top_data.csv") for complex in complexes]

In [ ]:
results = defaultdict(list)
for c in range(4):
    for id in tqdm(set(complex_zinc_ids[c])):
        results[c].append(zinc_id_pdbqt[zinc_id_total_dict[id]])

In [ ]:
# 保存pdbqt.gz文件
import random

for c in range(4):
    write_pdbqt_to_gz(random.sample(results[c], 10_000), f"{complexes[c]}_top_data_10k.pdbqt.gz")

## 索引每一类的代表分子

In [ ]:
# 用于索引每一类代表的分子结构
from collections import namedtuple

Cluster = namedtuple("Cluster", ["file_name", "zinc_ids"])

rep_file = "../dock_interaction_analysis/cluster_representation_for_interaction_analysis.txt"
f = open(rep_file, "r")
lines = f.read()
f.close()
analysis_file_content = []
cmpx_clusters = lines.strip().split("\n\n")
for cluster_str in cmpx_clusters:
    clusters = cluster_str.strip().split("\n")
    cmpx = clusters[0]
    for i, zinc_id_str in enumerate(clusters[1:]):
        zinc_ids = zinc_id_str.strip().split(" ")
        analysis_file_content.append(Cluster(f"{cmpx}_cluster_{i}.pdbqt.gz", zinc_ids))
len(analysis_file_content)

In [ ]:
for c in analysis_file_content:
    analysis_out_pdbqt_list = []
    for id in c.zinc_ids:
        analysis_out_pdbqt_list.append(zinc_id_pdbqt[zinc_id_total_dict[int(id[4:])]])
    write_pdbqt_to_gz(analysis_out_pdbqt_list, c.file_name)